# Parse SD monthly voter registration report

Uses:
- `requests` to get the PDF from [the Secretary of State's website](https://sdsos.gov/elections-voting/upcoming-elections/voter-registration-totals/voter-registration-by-county.aspx)
- `pdfplumber` to parse and validate the data

In [517]:
from io import BytesIO
from datetime import datetime
import csv

import requests
import pdfplumber

In [911]:
url = 'https://sdsos.gov/elections-voting/assets/StatewideVotersByCounty_11.1.2019.pdf'
req = requests.get(url)

In [912]:
pdf = pdfplumber.open(BytesIO(req.content))

In [913]:
# assumes a 2-page PDF with a table spanning both pages
if len(pdf.pages) != 2:
    raise Exception(f'Report page count is {len(pdf.pages)}, not 2 -- check it out: {url}')

In [914]:
lines = pdf.pages[0].extract_table() + pdf.pages[1].extract_table()

# guarding against some PDFs parsed with an extra column
lines = [x[:-1] if not x[-1] else x for x in lines]

In [915]:
# report date is in the first line
report_date = lines.pop(0)

In [916]:
report_date = report_date[0].split('-')[-1].strip()
report_date = datetime.strptime(report_date, '%B %d, %Y').date().isoformat()

print(report_date)

2019-11-01


In [917]:
# standardizing column names
column_fixes = {
    'oth': 'other',
    'no party\naffiliation': 'npa',
    'no labels': 'no_labels',
    'npa/ind': 'npa_ind'
}

In [918]:
# headers are in the first line of the data table
headers = lines.pop(0)
headers = [column_fixes.get(x.lower(), x.lower()) for x in headers]

In [919]:
headers

['county',
 'libertarian',
 'republican',
 'democratic',
 'npa_ind',
 'other',
 'inactive',
 'total active']

In [920]:
data = []

# loop over the remaining lines
for line in lines:

    # skip lines if repeated headers
    if 'statewide' in line[0].lower() or 'county' in line[0].lower():
        continue

    # the 'Total' line in the table means we're done
    if 'total' in line[0].lower():
        break

    # check that the number of values matches the number of headers
    assert(len(line) == len(headers))

    # zip it up into a dict
    county_data = dict(zip(headers, line))

    # delete the aggregate value totaling everything up
    del county_data['total active']

    # turn the numbers into integers
    for key in county_data:
        if key == 'county' or not county_data[key]:
            continue

        county_data[key] = int(county_data[key].replace(',', ''))

    # add the report date
    county_data['date'] = report_date
    data.append(county_data)

In [921]:
# doublecheck: 66 counties in SD
assert(len(data) == 66)

In [922]:
data[0]

{'county': 'Aurora',
 'libertarian': 1,
 'republican': 804,
 'democratic': 705,
 'npa_ind': 268,
 'other': 2,
 'inactive': 106,
 'date': '2019-11-01'}

In [923]:
# shift 'date' to the beginning of headers list for the output CSV
csv_headers = list(data[0].keys())
csv_headers = csv_headers[-1:] + csv_headers[:-1]

In [924]:
# write to file
filepath = f'data/{report_date}.csv'

with open(filepath, 'w') as outfile:
    writer = csv.DictWriter(
        outfile,
        fieldnames=csv_headers
    )

    writer.writeheader()
    writer.writerows(data)

    print(f'Wrote file: {filepath}')

Wrote file: data/2019-11-01.csv
